# AmpliMAP

## 1. Setting up the environment

In [ ]:
# Make neccessary directories
mkdir qza qzv lefse ancom

In [ ]:
# Activate the Qiime2 conda environment
conda activate qiime2

In [ ]:
# Check the Qiime2 version
qiime --version

## 2. Import and visualize the data

In [ ]:
# Visualize the metadata file
qiime metadata tabulate \
  --m-input-file metadata.tsv \
  --o-visualization qzv/metadata.qzv

In [ ]:
# view the metadata.qzv file
qiime tools view qzv/metadata.qzv

In [ ]:
# Import the raw data into Qiime2 using the manifest file
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path manifest.tsv \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path qza/paired-end-demux.qza

In [ ]:
# Visualize the imported data
qiime demux summarize \
  --i-data qza/paired-end-demux.qza \
  --p-n 100000 \
  --o-visualization qzv/paired-end-demux.qzv

In [ ]:
# view the demux.qzv file
qiime tools view qzv/demux.qzv

## 3. Quality control and feature table generation

In [ ]:
# Denoise the sequences
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs qza/paired-end-demux.qza \
  --p-trim-left-f 0 \
  --p-trim-left-r 0 \
  --p-trunc-len-f 0 \
  --p-trunc-len-r 0 \
  --p-n-threads 60 \
  --verbose \
  --o-representative-sequences qza/rep-seqs.qza \
  --o-table qza/table.qza \
  --o-denoising-stats qza/denoise-stats.qza

In [ ]:
# Visualize the denoising statistics
qiime metadata tabulate \
    --m-input-file qza/denoise-stats.qza \
    --o-visualization qzv/denoise-stats.qzv

In [ ]:
# view the denoise-stats.qzv file
qiime tools view qzv/denoise-stats.qzv

In [ ]:
# Visualize the feature table
qiime feature-table summarize \
  --i-table qza/table.qza \
  --m-sample-metadata-file bangladesh-ishtiaque-zeshan-anis-metadata_ninth_revision.tsv \
  --o-visualization qzv/table.qzv

In [ ]:
# view the table.qzv file
qiime tools view qzv/table.qzv

In [ ]:
# Visualize the representative sequences
qiime feature-table tabulate-seqs \
  --i-data qza/rep-seqs.qza \
  --o-visualization qzv/rep-seqs.qzv

In [ ]:
# view the rep-seqs.qzv file
qiime tools view qzv/rep-seqs.qzv

## 4. Phylogenetic analysis

In [ ]:
# Download the reference database
wget \
  -O "sepp-refs-gg-13-8.qza" \
  "https://data.qiime2.org/2021.4/common/sepp-refs-gg-13-8.qza"

In [ ]:
# Generate a sepp_tree
qiime fragment-insertion sepp \
  --i-representative-sequences qza/rep-seqs.qza \
  --i-reference-database sepp-refs-gg-13-8.qza \
  --o-tree qza/sepp_tree.qza \
  --o-placements qza/sepp_tree_placements.qza \
  --p-threads 60
  --verbose

In [ ]:
# Export the sepp tree
qiime tools export \
  --input-path qza/sepp_tree.qza \
  --output-path sepp_exported-tree

## 5. Diversity analysis

### Core metrics

In [ ]:
# Generate core matrics
qiime diversity core-metrics-phylogenetic \
  --i-table qza/table.qza \
  --i-phylogeny qza/sepp_tree.qza \
  --m-metadata-file bangladesh-ishtiaque-zeshan-anis-metadata_ninth_revision.tsv \
  --p-sampling-depth 3525 \
  --p-n-jobs-or-threads 'auto' \
  --output-dir core-metrics-results

### Alpha rarefaction

In [ ]:
#Visualize the rarefied feature table
qiime feature-table summarize \
  --i-table core-metrics-results/rarefied_table.qza \
  --m-sample-metadata-file metadata.tsv \
  --o-visualization qzv/rarefied_table.qzv

In [ ]:
# view the rarefied_table.qzv file
qiime tools view qzv/rarefied_table.qzv

### Alpha Diversity

In [ ]:
# Calculate Observed features
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/observed_features_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/observed_features.qzv

In [ ]:
# Calculate Shannon diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/shannon_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/shannon_diversity.qzv

In [ ]:
# Calculate faith phylogenetic diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/faith_pd.qzv

In [ ]:
# Calculate pielou's evenness
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics-results/evenness_vector.qza \
  --m-metadata-file metadata.tsv \
  --o-visualization core-metrics-results/pielou_evenness.qzv

### Beta diversity

In [ ]:
# Calculate Jaccard distance
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/jaccard_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Population \
  --o-visualization qzv/jaccard-distance-population.qzv \
  --p-pairwise

In [ ]:
# Calculate Bray Curtis
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Population \
  --o-visualization qzv/bray-curtis-population.qzv \
  --p-pairwise

In [ ]:
# Calculate Unweighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Population \
  --o-visualization qzv/unweighted-unifrac-population.qzv \
  --p-pairwise

In [ ]:
# Calculate Weighted UniFrac
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file metadata.tsv \
  --m-metadata-column Population \
  --o-visualization qzv/weighted-unifrac-population.qzv \
  --p-pairwise

### Beta Rarefaction

In [ ]:
# Perform beta rarefaction
qiime diversity beta-rarefaction \
    --i-table qza/table.qza \
    --p-metric weighted_unifrac \
    --p-clustering-method upgma \
    --m-metadata-file metadata.tsv \
    --p-sampling-depth 34568 \
    --i-phylogeny qza/rooted-tree.qza \
    --o-visualization qzv/beta-rarefaction.qzv \
    --verbose

In [ ]:
# Export the upgma tree
qiime tools export \
    --input-path qzv/beta-rarefaction.qzv \
    --output-path qzv/upgma-tree

## 6. Taxonomic analysis

In [ ]:
# Download the classifier
wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2021.4/common/gg-13-8-99-515-806-nb-classifier.qza"

In [ ]:
# Generate taxonomy
qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads qza/rep-seqs.qza \
  --o-classification qza/taxonomy.qza

In [ ]:
# Visualize the taxonomy
qiime metadata tabulate \
  --m-input-file qza/taxonomy.qza \
  --o-visualization qzv/taxonomy.qzv

In [ ]:
# view the taxonomy.qzv file
qiime tools view qzv/taxonomy.qzv

### Barplot for taxonomic composition

In [ ]:
# Filter out samples with fewer sequences than the rarefaction depth
qiime feature-table filter-samples \
  --i-table qza/table.qza \
  --p-min-frequency 34568 \
  --o-filtered-table qza/table_34568.qza

In [ ]:
# Create taxonomy barplot
qiime taxa barplot \
  --i-table qza/table_34568.qza \
  --i-taxonomy qza/taxonomy.qza \
  --m-metadata-file ./metadata.tsv \
  --o-visualization qzv/taxa_barplot.qzv

## 7. Differential abundance calculation with Lefse

In [ ]:
mkdir lefse

In [ ]:
# Collapse the feature table at genus level(level-6) for Lefse analysis
qiime taxa collapse \
--i-table qza/table.qza \
--o-collapsed-table lefse/feature-table-for-lefse-level-6-genus.qza \
--p-level 6 \
--i-taxonomy qza/taxonomy.qza

In [ ]:
# Filter out the very lowly abundant features
qiime feature-table filter-features \
  --i-table lefse/feature-table-for-lefsee-level-6-genus.qza \
  --p-min-samples 27 \
  --o-filtered-table lefse/filtered-feature-table-for-lefse-level-6-genus.qza

In [ ]:
# Visualize the filtered feature table for lefse
qiime feature-table summarize \
  --i-table lefse/filtered-feature-table-for-lefse-level-6-genus.qza \
  --m-sample-metadata-file metadata.tsv \
  --o-visualization lefse/filtered-feature-table-for-lefse-level-6-genus.qzv

In [ ]:
# view the filtered-feature-table-for-lefse-level-6-genus.qzv file
qiime tools view lefse/filtered-feature-table-for-lefse-level-6-genus.qzv

In [ ]:
# Calculate relative frequency from the filtered featured table
qiime feature-table relative-frequency \
--i-table lefse/filtered-feature-table-for-lefse-level-6-genus.qza \
--o-relative-frequency-table lefse/relative-frequency.qza

In [ ]:
# Convert the relative frequency into a biome file
qiime tools export \
    --input-path lefse/relative-frequency.qza \
    --output-path lefse/biom_file

In [ ]:
# Convert biome file to .tsv file
biom convert \
    -i lefse/biom_file/feature-table.biom \
    -o lefse/biom_file/feature-table.tsv \
    --to-tsv

In [ ]:
# Convert biom file to .txt file
biom convert \
    -i lefse/biom_file/feature-table.biom \
    -o lefse/biom_file/frequency.table.txt \
    --header-key “taxonomy” --to-tsv

In [ ]:
#Replace “;” with “|”
cat lefse/biom_file/feature-table.tsv | \
tr ";" "|" > lefse/relative-frequency-table.tsv

**At this point, run the lefse_input_genarator.sh script. Before running the script, make sure the order of Sample-IDs (i.e. rownames) in the metadata file exactly matches the order of the sample-IDs (i.e. column names left to right) in the relative-frequency-table.tsv file.**

In [ ]:
pwd

In [ ]:
cd lefse

In [ ]:
# Run Lefse
for name in Population

do

#Input file name
input=$name/${name}-lefse-input.tsv

#Normalization value (chosen from filtered-feature-table-lefse.qzv file)
nvalue=41597

#Image DPI
dpi=600

lefse_format_input.py \
    $input \
    $name/temp1.in \
    -c 1 -u 2 -o $nvalue

lefse_run.py \
    $name/temp1.in \
    $name/temp2.res
    
lefse_plot_res.py \
    $name/temp2.res \
    $name/${name}-lefse.svg \
    --feature_font_size 11 \
    --format svg \
    --title $name \
    --dpi $dpi \
    --left_space .25


lefse_plot_cladogram.py \
    $name/temp2.res \
    $name/${name}-cladogram-lefse.svg \
    --title $name \
    --format svg \
    --dpi $dpi
    

# Remove intermediate files
rm -f $name/temp*

done

## 8. Differential abundance calculation with ANCOM

In [ ]:
mkdir ancom

In [ ]:
# Collapse the feature table at genus level(6) for ANCOM
qiime taxa collapse \
  --i-table qza/table.qza \
  --i-taxonomy qza/taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table ancom/feature-table-for-ancom-level-6-genus.qza

In [ ]:
# Filter out the very lowly abundant features
qiime feature-table filter-features \
  --i-table ancom/feature-table-for-ancom-level-6-genus.qza \
  --p-min-frequency 26 \
  --p-min-samples 10 \
  --o-filtered-table ancom/filtered-feature-table-for-ancom-level-6-genus.qza

In [ ]:
# Generate the composition of the filtered feature table
qiime composition add-pseudocount \
  --i-table ancom/filtered-feature-table-for-ancom-level-6-genus.qza \
  --o-composition-table ancom/composition-of-filtered-feature-table-for-ancom-level-6-genus.qza

### Population based differential abundance calculation

In [ ]:
# Differentially expressed features in Bengali population
qiime composition ancom \
  --i-table ancom/composition-of-filtered-feature-table-for-ancom-level-6-genus.qza \
  --m-metadata-file metadata_for_ancom.tsv \
  --m-metadata-column Bengali_cohort \
  --verbose \
  --o-visualization ancom/ancom-Bengali.qzv

In [ ]:
# Differentially abundant features in Chakma population
qiime composition ancom \
  --i-table ancom/composition-of-filtered-feature-table-for-ancom-level-6-genus.qza \
  --m-metadata-file metadata_for_ancom.tsv \
  --m-metadata-column Chakma_cohort \
  --verbose \
  --o-visualization ancom/ancom-Chakma.qzv

In [ ]:
# Differentially abundant features in Khiyang population
qiime composition ancom \
  --i-table ancom/composition-of-filtered-feature-table-for-ancom-level-6-genus.qza \
  --m-metadata-file metadata_for_ancom.tsv \
  --m-metadata-column Khiyang_cohort \
  --verbose \
  --o-visualization ancom/ancom-Khiyang.qzv

In [ ]:
# Differentially abundant features in Tripura population
qiime composition ancom \
  --i-table qza/composition-of-filtered-feature-table-for-ancom-level-6-genus.qza \
  --m-metadata-file metadata_for_ancom.tsv \
  --m-metadata-column Tripura_cohort \
  --verbose \
  --o-visualization ancom/ancom-Tripura.qzv

In [ ]:
# Differentially abundant features in Marma population
qiime composition ancom \
  --i-table qza/composition-of-filtered-feature-table-for-ancom-level-6-genus.qza \
  --m-metadata-file metadata_for_ancom.tsv \
  --m-metadata-column Marma_cohort \
  --verbose \
  --o-visualization ancom/ancom-Marma.qzv